In [11]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm
from datetime import datetime, timedelta
df = pd.read_csv("russell-2000-index-08-25-2024.csv")
us2000_1000 = list(df['Symbol'].values)
df_list = []
for stock in tqdm(us2000_1000):
    df = yf.download(stock, interval='5m', start=datetime.now() - timedelta(60), progress=False, rounding=2)
    df['code'] = stock
    df_list.append(df)
df = yf.download("^RUT", interval='5m', start=datetime.now() - timedelta(60), progress=False, rounding=2)
df['code'] = "US2000"
df_list.append(df)
final = pd.concat(df_list)
final['datetime'] = final.index
pivoted = final.pivot(columns='code', values='Close', index='datetime')
pivoted.to_parquet("US2000_top1000.parquet")

 44%|████▍     | 439/1000 [02:10<02:47,  3.34it/s]
1 Failed download:
['CRD.A']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
 53%|█████▎    | 531/1000 [02:39<02:19,  3.37it/s]
1 Failed download:
['DNMRW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
 73%|███████▎  | 728/1000 [03:36<01:20,  3.39it/s]
1 Failed download:
['GEF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (5m 2024-06-26 16:33:03.080678 -> 2024-08-25 04:33:03-04:00)')
100%|██████████| 1000/1000 [04:59<00:00,  3.33it/s]
/var/folders/cb/s9sjq6dx01990v5hdvfhl7x40000gn/T/ipykernel_14439/3127584225.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  final = pd.concat(df_list)


In [24]:
def backtest_code(res, code, benchmark = 2):
    rolling = 5
    benchmark_index = "US2000"
    
    pivoted = res[[code, benchmark_index]]
    pivoted.ffill()
    ret = pivoted.pct_change(fill_method=None)
    try:
        ret['stock_price'] = pivoted[code]
        ret['ret_diff'] = ret[code] - ret[benchmark_index]
        # type 1
        # ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std() * 100
        # ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean() * 100
        # ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])
        
        # type 2
        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

        ret[code] = (1 + ret[code]).cumprod() - 1

        ret['index'] = [i for i in range(len(ret))]

        # check direction
        ret['lag10'] = ret['stock_price'].shift(-4)
        ret['change_stock'] = (ret['lag10'] - ret[code])/ret[code]
        ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)

        # check if the direction are the same, if not, tned to not being a mean reversion
        ret['lag10_twii'] = ret[benchmark_index].shift(-4)
        ret['change_twii'] = (ret['lag10_twii'] - ret[benchmark_index])/ret[benchmark_index]
        ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
        ret['filter_2'] = ret.apply(lambda x: 0 if abs(x['change_stock']) < abs(x['change_twii']) else 1, axis=1)

        # check raise
        ret[f'{code}_lag60'] = ret['stock_price'].shift(-12)
        ret['target_60mins'] = (ret[f'{code}_lag60'] - ret['stock_price'])/ret['stock_price'] * ret['direction']
        ret = ret[(ret['stdize_ret_diff'] >= benchmark) & (ret['filter_1'] == 1) & (ret['filter_2'] == 1)][['target_60mins', 'stdize_ret_diff']].dropna()
        ret['code'] = code
        return ret
    except:
        print(pivoted)

In [25]:
res = pd.read_parquet("US2000_top1000.parquet")
code_list=list(set(res.columns))

dfs = []
for code in tqdm(code_list[:]):
    if code == 'US2000': continue
    df = backtest_code(res, code)
    if len(df) == 0: continue
    dfs.append(df)

 17%|█▋        | 168/998 [00:07<00:37, 22.28it/s]/var/folders/cb/s9sjq6dx01990v5hdvfhl7x40000gn/T/ipykernel_14439/2140694297.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
 35%|███▌      | 352/998 [00:15<00:30, 21.16it/s]/var/folders/cb/s9sjq6dx01990v5hdvfhl7x40000gn/T/ipykernel_14439/2140694297.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
 49%|████▉     | 490/998 [00:22<00:25, 20.01it/s]/var/folders/cb/s9sjq6dx01990v5hdvfhl7x40000gn/T/ipykernel_14439/2140694297.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
100%|██████████| 998/998 [00:44<00:00

In [28]:
final

code,target_60mins,stdize_ret_diff,code
datetime,,,
2024-07-11 09:50:00-04:00,-0.021527,2.440304,EXPI
2024-08-01 10:25:00-04:00,0.003509,3.351977,EXPI
2024-08-08 09:55:00-04:00,0.018440,3.261075,DCGO
2024-06-28 15:55:00-04:00,0.093104,3.952690,CMPX
2024-07-01 09:30:00-04:00,0.001325,30.054927,CMPX
...,...,...,...
2024-08-06 09:35:00-04:00,0.004480,3.338703,JANX
2024-08-08 09:30:00-04:00,0.011062,3.377882,JANX
2024-08-08 09:35:00-04:00,-0.000286,3.558728,JANX


In [27]:
import numpy as np
import pandas as pd
from scipy import stats
# to test if return after 60 mins are greater than {test}
final = pd.concat(dfs)
print(len(final))
final = final[final['target_60mins'] != 0.0]
for test in range(0, 25, 1):
    test /= 1000
    final = pd.concat(dfs)
    final = final[final['target_60mins'] != 0.0]
    final['target_60mins'] = final['target_60mins'] - test
    mean = final['target_60mins'].mean()
    n = len(final)
    s = final['target_60mins'].std()

    t_statistic = (mean - 0.0) / (s / np.sqrt(n))

    t_stat, p_value = stats.ttest_1samp(final['target_60mins'], 0.0)

    if t_stat > 0:
        one_tailed_p_value = p_value / 2
    else:
        one_tailed_p_value = 1 - (p_value / 2)

    print(f"{test} P-value: {np.round(one_tailed_p_value, 2)} {np.round(t_statistic, 2)}")
print("US100,", len(final))

6400
0.0 P-value: 0.0 42.06
0.001 P-value: 0.0 40.04
0.002 P-value: 0.0 38.01
0.003 P-value: 0.0 35.99
0.004 P-value: 0.0 33.96
0.005 P-value: 0.0 31.94
0.006 P-value: 0.0 29.92
0.007 P-value: 0.0 27.89
0.008 P-value: 0.0 25.87
0.009 P-value: 0.0 23.84
0.01 P-value: 0.0 21.82
0.011 P-value: 0.0 19.8
0.012 P-value: 0.0 17.77
0.013 P-value: 0.0 15.75
0.014 P-value: 0.0 13.72
0.015 P-value: 0.0 11.7
0.016 P-value: 0.0 9.68
0.017 P-value: 0.0 7.65
0.018 P-value: 0.0 5.63
0.019 P-value: 0.0 3.6
0.02 P-value: 0.06 1.58
0.021 P-value: 0.67 -0.44
0.022 P-value: 0.99 -2.47
0.023 P-value: 1.0 -4.49
0.024 P-value: 1.0 -6.52
US100, 6298
